# 02wk-005: 타이타닉, Autogluon

최규빈  
2023-09-12

# 1. 강의영상

<https://youtu.be/playlist?list=PLQqh36zP38-zZrOGpLc8spPa9L39RiNhR&si=TFl5m9-VohYT_47L>

# 2. Import

``` python
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
```

    /kaggle/input/titanic/train.csv
    /kaggle/input/titanic/test.csv
    /kaggle/input/titanic/gender_submission.csv

In [3]:
#pip install autogluon

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor

# 3. 분석의 절차

## A. 데이터

`-` 비유: 문제를 받아오는 과정으로 비유할 수 있다.

In [ ]:
tr = TabularDataset("/kaggle/input/titanic/train.csv")
tst = TabularDataset("/kaggle/input/titanic/test.csv")

## B. Predictor 생성

`-` 비유: 문제를 풀 학생을 생성하는 과정으로 비유할 수 있다.

In [3]:
predictr = TabularPredictor("Survived")

No path specified. Models will be saved in: "AutogluonModels/ag-20230910_162629/"

## C. 적합(fit)

`-` 비유: 학생이 공부를 하는 과정으로 비유할 수 있다.

`-` 학습

In [4]:
predictr.fit(tr) 
# 학생(predictr)에게 문제(tr)를 줘서 학습을 시킴(predictr.fit())

Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels/ag-20230910_162629/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #26~22.04.1-Ubuntu SMP PREEMPT_DYNAMIC Thu Jul 13 16:27:29 UTC 2
Disk Space Avail:   306.27 GB / 490.57 GB (62.4%)
Train Data Rows:    891
Train Data Columns: 11
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
    2 unique label values:  [0, 1]
    If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
    Available Memory:                    126262.11 MB
    Train Data

`-` 리더보드확인 (모의고사 채점)

In [5]:
predictr.leaderboard()

                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.860335       0.193071  4.142821                0.000378           0.218161            2       True         14
1         LightGBMLarge   0.832402       0.003180  0.422871                0.003180           0.422871            1       True         13
2        NeuralNetTorch   0.832402       0.009243  1.488933                0.009243           1.488933            1       True         12
3              CatBoost   0.826816       0.003432  0.526762                0.003432           0.526762            1       True          7
4       NeuralNetFastAI   0.826816       0.007648  1.326021                0.007648           1.326021            1       True         10
5              LightGBM   0.821229       0.003042  0.157547                0.003042           0.157547            1       True          4
6            LightGBMXT   0.815642

## D. 예측 (predict)

`-` 비유: 학습이후에 문제를 푸는 과정으로 비유할 수 있다.

`-` training set 을 풀어봄 (predict) $\to$ 점수 확인

In [6]:
(tr.Survived == predictr.predict(tr)).mean()

In [7]:
(tr.Survived == (tr.Sex == "female")).mean() # 예전점수와 비교

`-` test set 을 풀어봄 (predict) $\to$ 점수 확인 하러 캐글에 결과제출

In [ ]:
tst.assign(Survived = predictr.predict(tst)).loc[:,['PassengerId','Survived']]\
.to_csv("autogluon_submission.csv",index=False)